ElasticSearch - это документарная база данных с возможностью полнотекстовго и географического поиска.

Попробуем решить с ее помощью знакомую задачу - выкачать NPlus1 и положить статьи в базу данных.

База данных [ставится](https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html) довольно просто, дальше к ней можно обращаться просто по локальному адресу, порт 9200: https://127.0.0.1:9200/ . На все запросы база данных будет присылать JSON-документ. То есть при желании с ней можно работать при помощи библиотеки `requests`. Вместо этого поставим библиотеку `elasticsearch` и будем работать через нее.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import datetime
from tqdm import tqdm

# Импортируем библиотеку для БД.
from elasticsearch import Elasticsearch


К базе данных нужно обращаться при помощи класса `Elasticsearch`. 

In [2]:
# Подключаемся к базе данных.
es=Elasticsearch()

In [3]:
es

<Elasticsearch([{}])>

Создадим класс `NPlus1Article`, который будет хранить информацию по статье и преобразовывать ее к словарю или JSON. Используем старые функции, которые выгружают статьи за весь день и статьи за заданный период.

In [11]:
delcom=re.compile("<!--.+-->", re.S)

# Класс, хранящий информацию о статье.
class NPlus1Article:
    def __init__(self):
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        self.head=""
        self.text=""
        
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "rubrics":"'+self.rubr+'", "difficulty":"'
        res+=self.diff+'", "title":"'+self.head+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "difficulty":self.diff,\
             "title":self.head, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res
    
def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    #print(r.text)
    art = NPlus1Article()
    tables = re.split("</div>",re.split('="tables"', r.text)[1])[0]
    t1 = re.split("</time>", re.split("<time", tables)[1])[0]
    art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
    art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
    art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
    art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
    art.head = re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

    beaux_text = BeautifulSoup(art.text, "html.parser")
    art.text = delcom.sub("", beaux_text.get_text() )
    art.text = art.text.replace('\xa0', ' ')

    # print(art.n_time, art.n_date, art.n_rubr, art.n_diff)
    # print(art.n_head)
    # print(art.n_author)
    # print(art.n_text)
    #return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
    return art

def getDayArticles(adr):
    r = requests.get(adr)
    titles = BeautifulSoup(r.text, "html.parser")("article")
    #print(titles)
    addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
    #print(addrs)
    articles = []
    for adr in addrs:
        articles.append(getArticleTextNPlus1(adr))
    return articles

Загрузим статьи за несколько дней.

In [21]:
#arts=getDayArticles("https://nplus1.ru/news/2019/02/01")
arts=getDayArticles("https://nplus1.ru/news/2017/10/16/")
#arts=getDayArticles("https://nplus1.ru/news/2016/08/04/")

In [9]:
!pip install --upgrade bs4

Requirement already up-to-date: bs4 in c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages (0.0.1)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Вся информация хранится в так называемых индексах (аналог базы данных). 

Создадим индекс при помощи `indices.create(index=название_индекса)`

In [17]:
es.indices.create(index="nplus1")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nplus1'}

Для индекса можно задать тип документа, который будет в нем храниться. Если тип не будет задан, БД сама создаст его при первой загрузке документа (не всегда так, как этого хотелось бы нам, поэтому лучше тип задавать явно).

Описание типа (маппинг) ведется в виде JSON или словаря. Оно должно содержать в себе название типа (в нашем случае это "article") и описание полей. Описание полей дается после ключевого слова "properties". Для каждого поля задается его название за которым следует описание поля: тип, формат, анализатор (язык), другие параметры.

Маппинг для существующего индекса создается при помощи `indices.put_mapping`, в [который](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.put_mapping) передается названия индекса и типа. Можно передать маппинг сразу в функцию создания индекса.

In [18]:
mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":"HH:mm"},
                                "difficulty":{"type":"double"},
                                "rubrics":{"type":"text", "analyzer":"russian"},
                                "text":{"type":"text", "analyzer":"russian"},
                                "title":{"type":"text", "analyzer":"russian"}}}}

es.indices.put_mapping(index="nplus1", doc_type='article', body=mapit, include_type_name=True)

{'acknowledged': True}

При желании индекс можно [удалить](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.delete).

In [14]:
es.indices.delete("nplus1")

{'acknowledged': True}

Можно получить список индексов для текущей базы.

In [19]:
es.indices.get_alias("*")

{'nplus1': {'aliases': {}}}

Можно получить маппинг для интересующего нас индекса.

In [20]:
es.indices.get_mapping(index="nplus1", doc_type='article', include_type_name=True)

{'nplus1': {'mappings': {'article': {'properties': {'author': {'type': 'text'},
     'date': {'type': 'text'},
     'difficulty': {'type': 'double'},
     'rubrics': {'type': 'text', 'analyzer': 'russian'},
     'text': {'type': 'text', 'analyzer': 'russian'},
     'time': {'type': 'date', 'format': 'HH:mm'},
     'title': {'type': 'text', 'analyzer': 'russian'}}}}}}

Загрузка документов в базу ведется при помощи функции `index`. Тем самым БД и библиотека подчеркивают, что для них строится полнотекстовый индекс для быстрого поиска. [В функцию](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.index) передаются название индекса и типа документа. Также можно передать идентификатор документа и другие параметры.

In [22]:
%%time

for art in tqdm(arts):
    es.index(index="nplus1", doc_type='article', body=art.toDict())

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 69.39it/s]

Wall time: 273 ms


Получить документ по его идентификатору можно при помощи [функции](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.get) `get`.

Но в дальнейшем мы будем пользоваться [функцией](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.search) `search`, которая позволяет проводить любой поиск, фильтрацию и аггрегацию информации. Для поиска надо передать название индекса и запрос в виде JSON или словаря.

В простейшем случае запрос содержит в себе поле `query`, включающее в себе описание документов, которые мы хотим найти. Также в запрос могут включаться другие параметры: количество выводимых документов, фильтры, аггрегация и др.

В запросе ниже мы используем `match_all`, то есть просьбу вывести все документы. По умолчанию выводятся 10 документов.

Результат содержит в себе несколько полей, `["hits"]["hits"]` содержит найденные документы.

In [23]:
#es.get(index="nplus1", doc_type='article', id=1)
res = es.search(index="nplus1", body={"query": {"match_all": {}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'dAW2FHABiLWAMI7vjS0T',
  '_score': 1.0,
  '_source': {'date': '16 Окт. 2017',
   'time': '18:33',
   'rubrics': 'Медицина',
   'difficulty': '2.1',
   'title': 'В донорской крови нашли африканский птичий вирус',
   'author': 'Дарья Спасская',
   'text': '\n\n\nАфриканский вирус Усуту, который с начала 2000-х годов вызвал\nв Европе несколько эпидемий среди птиц, встречается у людей чаще, чем\nожидалось. В частности, вирус был обнаружен в семи образцах донорской крови в\nавстрийском донорском центре. Об этом сообщает европейский эпидемиологический\nжурнал Eurosurveillance, а\nо распространении вируса среди птиц на территории Австрии и Венгрии авторы\nрассказали в журнале Emerging Microbes & Infections.\nВирус Усуту\nотносится к группе флавивирусов, так же как вирус энцефалита, лихорадки\nЗападного Нила, денге и вирус Зика. Как и многие представители этой группы,\nпереносится вирус Усуту комарами. Он был впервые описан на территории Аф

Сам по себе запрос содержит информацию о шардах, найденные документы, их количество, информацию о выполнении запроса.

In [24]:
res

{'took': 234,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 18, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'nplus1',
    '_type': 'article',
    '_id': 'dAW2FHABiLWAMI7vjS0T',
    '_score': 1.0,
    '_source': {'date': '16 Окт. 2017',
     'time': '18:33',
     'rubrics': 'Медицина',
     'difficulty': '2.1',
     'title': 'В донорской крови нашли африканский птичий вирус',
     'author': 'Дарья Спасская',
     'text': '\n\n\nАфриканский вирус Усуту, который с начала 2000-х годов вызвал\nв Европе несколько эпидемий среди птиц, встречается у людей чаще, чем\nожидалось. В частности, вирус был обнаружен в семи образцах донорской крови в\nавстрийском донорском центре. Об этом сообщает европейский эпидемиологический\nжурнал Eurosurveillance, а\nо распространении вируса среди птиц на территории Австрии и Венгрии авторы\nрассказали в журнале Emerging Microbes & Infections.\nВирус Усуту\nотносится к гру

В качестве описания документов можно передать более конкретное описание, задаваемое ключевым словом `match`. Ниже мы просим найти все документы, поле `title` которых содержит слово `пустота`.

Если теперь вернуться к описанию типа (маппингу), то мы увидим, что поле `title` задавалось типом `text` с русским анализатором. Это означает, что все слова в документе будут приведены к начальной форме и поиск по ним будет вестись вне зависимости от формы слова. Если мы забудем задать русский язык, то по умолчанию возьмется английский, то есть нормального ндексирования и поиска не получится. Если бы хотели искать по точному совпадению значения поля, то следовало бы использовать тип `keyword`. Также можно использовать другие типы: `integer`, `float`, `date` и другие. Для всех полей можно задать форматирование, которое зависит от типа данных. В нашем случае мы задали, что время содержит в себе часы и минуты.

In [32]:
res = es.search(index="nplus1", body={"query": {"match": {"title": "донорская"}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'dAW2FHABiLWAMI7vjS0T',
  '_score': 2.6472712,
  '_source': {'date': '16 Окт. 2017',
   'time': '18:33',
   'rubrics': 'Медицина',
   'difficulty': '2.1',
   'title': 'В донорской крови нашли африканский птичий вирус',
   'author': 'Дарья Спасская',
   'text': '\n\n\nАфриканский вирус Усуту, который с начала 2000-х годов вызвал\nв Европе несколько эпидемий среди птиц, встречается у людей чаще, чем\nожидалось. В частности, вирус был обнаружен в семи образцах донорской крови в\nавстрийском донорском центре. Об этом сообщает европейский эпидемиологический\nжурнал Eurosurveillance, а\nо распространении вируса среди птиц на территории Австрии и Венгрии авторы\nрассказали в журнале Emerging Microbes & Infections.\nВирус Усуту\nотносится к группе флавивирусов, так же как вирус энцефалита, лихорадки\nЗападного Нила, денге и вирус Зика. Как и многие представители этой группы,\nпереносится вирус Усуту комарами. Он был впервые описан на террито

In [28]:
res

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 2.6472712,
  'hits': [{'_index': 'nplus1',
    '_type': 'article',
    '_id': 'dAW2FHABiLWAMI7vjS0T',
    '_score': 2.6472712,
    '_source': {'date': '16 Окт. 2017',
     'time': '18:33',
     'rubrics': 'Медицина',
     'difficulty': '2.1',
     'title': 'В донорской крови нашли африканский птичий вирус',
     'author': 'Дарья Спасская',
     'text': '\n\n\nАфриканский вирус Усуту, который с начала 2000-х годов вызвал\nв Европе несколько эпидемий среди птиц, встречается у людей чаще, чем\nожидалось. В частности, вирус был обнаружен в семи образцах донорской крови в\nавстрийском донорском центре. Об этом сообщает европейский эпидемиологический\nжурнал Eurosurveillance, а\nо распространении вируса среди птиц на территории Австрии и Венгрии авторы\nрассказали в журнале Emerging Microbes & Infections.\nВирус Усуту\nотноси

Если мы хотим искать очень условный стем формы слова, следует использовать ключевое слово `term`. 

In [42]:
res = es.search(index="nplus1", body={"query": {"term": {"title": "африканск"}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'dAW2FHABiLWAMI7vjS0T',
  '_score': 2.6472712,
  '_source': {'date': '16 Окт. 2017',
   'time': '18:33',
   'rubrics': 'Медицина',
   'difficulty': '2.1',
   'title': 'В донорской крови нашли африканский птичий вирус',
   'author': 'Дарья Спасская',
   'text': '\n\n\nАфриканский вирус Усуту, который с начала 2000-х годов вызвал\nв Европе несколько эпидемий среди птиц, встречается у людей чаще, чем\nожидалось. В частности, вирус был обнаружен в семи образцах донорской крови в\nавстрийском донорском центре. Об этом сообщает европейский эпидемиологический\nжурнал Eurosurveillance, а\nо распространении вируса среди птиц на территории Австрии и Венгрии авторы\nрассказали в журнале Emerging Microbes & Infections.\nВирус Усуту\nотносится к группе флавивирусов, так же как вирус энцефалита, лихорадки\nЗападного Нила, денге и вирус Зика. Как и многие представители этой группы,\nпереносится вирус Усуту комарами. Он был впервые описан на террито

БД ищет любые слова из запроса за исключением стоп-слов. Порядок слов при этом не важен. Наличие слов в итоговом документе в итоге влияет на поле `_score`, высчитываемое на основе меры tf*idf.

In [43]:
res = es.search(index="nplus1", body={"query": {"match": {"text": "Германии и Франции"}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'ewW2FHABiLWAMI7vjS2r',
  '_score': 5.4853954,
  '_source': {'date': '16 Окт. 2017',
   'time': '14:57',
   'rubrics': 'Физика',
   'difficulty': '4.7',
   'title': 'Физики научились управлять движением топологических дефектов в ионных цепочках',
   'author': 'Александр Дубов',
   'text': '\n\n\nДвижением топологических дефектов в кристаллических структурах, которые образуются из ионов в ионных ловушках, можно управлять, изменяя амплитуду внешнего воздействия. Такой эффект группа физиков из Германии и Франции продемонстрировала на зигзагообразных ионных цепочках, состоящих из 34 ионов магния. Результаты исследования опубликованы в Physical Review Letters.\nТопологические дефекты в кристалле — области периодической кристаллической структуры, на которых происходит нарушение симметрии кристалла и одна топологическая фаза сменяется на другую. В трехмерных кристаллах топологическими дефектами являются, например, дислокации (при нарушении 

Если мы хотим найти все слова из запроса, к нему следует добавить параметр `operator`созначением `and`.

In [44]:
res = es.search(index="nplus1", body={"query": {"match": {"text": 
                                       {"query":"Франции Германии и", "operator": "and"}}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'ewW2FHABiLWAMI7vjS2r',
  '_score': 5.4853954,
  '_source': {'date': '16 Окт. 2017',
   'time': '14:57',
   'rubrics': 'Физика',
   'difficulty': '4.7',
   'title': 'Физики научились управлять движением топологических дефектов в ионных цепочках',
   'author': 'Александр Дубов',
   'text': '\n\n\nДвижением топологических дефектов в кристаллических структурах, которые образуются из ионов в ионных ловушках, можно управлять, изменяя амплитуду внешнего воздействия. Такой эффект группа физиков из Германии и Франции продемонстрировала на зигзагообразных ионных цепочках, состоящих из 34 ионов магния. Результаты исследования опубликованы в Physical Review Letters.\nТопологические дефекты в кристалле — области периодической кристаллической структуры, на которых происходит нарушение симметрии кристалла и одна топологическая фаза сменяется на другую. В трехмерных кристаллах топологическими дефектами являются, например, дислокации (при нарушении 

Также можно задавать интервалы поиска при помощи ключевого слова `range`. `gte` и `lt` задают операторы "больше или равно" и "меньше".

In [45]:
res = es.search(index="nplus1", body={"query": {"range": {"time": 
                                       {"gte":"10:00", "lt": "12:00"}}}})
res["hits"]["hits"]


[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'ggW2FHABiLWAMI7vjS3t',
  '_score': 1.0,
  '_source': {'date': '16 Окт. 2017',
   'time': '11:39',
   'rubrics': 'Энергетика',
   'difficulty': '3.7',
   'title': 'Исландцы заставили электростанцию улавливать углекислый газ',
   'author': 'Василий Сычев',
   'text': '\n\n\nИсландская энергетическая компания Reykjavik Energy совместно с швейцарской Climeworks провела модернизацию геотермальной электростанции Хедлисхейди около вулкана Хенгидль, установив на ней модули прямого улавливания диоксида углерода из воздуха. Как пишет Quartz, благодаря такому усовершенствованию исландская станция стала первой в мире электростанцией с отрицательным выбросом CO2.Выбросы углекислого газа считаются одной из нескольких возможных причин глобального потепления. Поэтому некоторые страны мира начали уделять внимание снижению уровня выброса CO2 в атмосферу. Это делается, в частности, поощрением граждан на использование электротранспорта, а также постепе

---

Теперь немного отвлечемся на замеры производительности ElasticSearch. Скорость ее работы с отдельными запросами оставляет желать лучшего - несколько десятков запросов в секунду. Гораздо лучше она работает при помощи потоковых запросов, отправляющих на запись сразу много документов.

Заметим, что при параллельной отправке документов БД работает быстрее, но до миллиона запросов в секунду всё равно далеко. Для того, чтобы получить такую скорость необходим кластер.

Попробуем положить в базу первый миллион сочетаний "существительное+прилагательное" из базы [КОСИКО](http://cosyco.ru/). Для этого создадим индекс и зададим тип документов.

In [11]:
es.indices.delete("cosyco")
es.indices.create(index = "cosyco")
mapit = {"combinations" : {"properties" : {"noun" : {"type" : "text", "analyzer" : "russian"},
                                           "adj"  : {"type" : "text", "analyzer" : "russian"},
                                           "frq"  : {"type": "integer"}}}}

es.indices.put_mapping(index = "cosyco", 
                       doc_type = 'combinations', 
                       body = mapit, 
                       include_type_name=True)

{'acknowledged': True}

Код ниже обеспечивает около 10 запросов в секунду - просто слезы.

In [5]:
fil = open("noun_adj_inf_combs.txt")
cntr = 0
for line in tqdm(fil):
    r = line.split(";")
    #r={"noun":r[0], "adj":r[1], "frq": r[2]}
    r = {"noun":r[0], "adj":r[1]}
    es.index(index="cosyco", doc_type='combinations', body=r)
    cntr += 1
    if cntr > 200:
        break
fil.close()

200it [00:22,  9.12it/s]

Создадим функции, которые будут читать данные из файла и класть их в базу. Так как мы хотим повышенную производительность, класть будем по нескоклько тысяч документов за один раз.

In [6]:
def getCosycoLines(start, size):
    fil = open("noun_adj_inf_combs.txt")
    for i in range(start):
        _ = fil.readline()

    res=[]
    for i in range(size):
        r = fil.readline().split(";")
        #r={"noun":r[0], "adj":r[1], "frq": r[2]}
        r2 = { "index": {"_index": "cosyco", "_type": "combinations"} }
        res.append(r2)
        r2 = {"noun":r[0].strip(), "adj":r[1].strip()}
        res.append(r2)
        
    fil.close()
    return res


In [7]:
rrr = getCosycoLines(0, 1000000)

In [8]:
%%time
#bulk(es, getCosycoLines(0, 1000000), index="cosyco", doc_type='combinations')
# Возвращает информацию об ошибках - кто был добавлен, а кто нет.
_ = es.bulk(rrr, index="cosyco", doc_type='combinations')


TransportError: TransportError(413, '')

Код ниже занял около 5 секунд: 20000 записей в секунду. Не так много, но гораздо лучше, чем было.

In [12]:
%%time

for i in tqdm(range(100)):
    passed=False
    while not passed:
        try:
            acq = es.bulk(getCosycoLines(i*10000, 10000), index="cosyco", doc_type='combinations')
            passed=True
        except:
            print(acq)



100%|██████████| 100/100 [01:43<00:00,  1.04s/it]

CPU times: user 44.6 s, sys: 1.01 s, total: 45.6 s
Wall time: 1min 43s


In [52]:
'Ё' < 'А'

True

Посмотрим как со скоростью поиска.

In [13]:
%%time
res = es.search(index="cosyco", body={"query": {"match_all": {}}})
res

CPU times: user 464 µs, sys: 3.93 ms, total: 4.39 ms
Wall time: 2.67 s


In [15]:
%%time
res = es.search(index="cosyco", body={"query": {"match": {"adj":"БЕЛЫЙ"}}})
print(res)

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 839, 'relation': 'eq'}, 'max_score': 7.082705, 'hits': [{'_index': 'cosyco', '_type': 'combinations', '_id': 'bNZHEnABtPFlvM7f_trQ', '_score': 7.082705, '_source': {'noun': 'АВТОМОБИЛЬЧИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': '-9ZHEnABtPFlvM7f_tzQ', '_score': 7.082705, '_source': {'noun': 'АВТОМОТРИСА', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'ldZHEnABtPFlvM7f_uPR', '_score': 7.082705, '_source': {'noun': 'АВТООТВЕТЧИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'ZNZHEnABtPFlvM7f_unR', '_score': 7.082705, '_source': {'noun': 'АВТОПОКРЫШКА', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'tNZHEnABtPFlvM7f_unR', '_score': 7.082705, '_source': {'noun': 'АВТОПОРТРЕТ', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'TNZHEnA

10 миллисекунд при поиске в миллионе документов - очень неплохо.

Хотя если вытаскивать все документы... Для этого надо добавить поле `size`.

In [16]:
%%time
res = es.search(index="cosyco", body={"size":1000, "query": {"match": {"adj":"белый"}}})
print(res)

{'took': 43, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 839, 'relation': 'eq'}, 'max_score': 7.082705, 'hits': [{'_index': 'cosyco', '_type': 'combinations', '_id': 'bNZHEnABtPFlvM7f_trQ', '_score': 7.082705, '_source': {'noun': 'АВТОМОБИЛЬЧИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': '-9ZHEnABtPFlvM7f_tzQ', '_score': 7.082705, '_source': {'noun': 'АВТОМОТРИСА', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'ldZHEnABtPFlvM7f_uPR', '_score': 7.082705, '_source': {'noun': 'АВТООТВЕТЧИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'ZNZHEnABtPFlvM7f_unR', '_score': 7.082705, '_source': {'noun': 'АВТОПОКРЫШКА', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'tNZHEnABtPFlvM7f_unR', '_score': 7.082705, '_source': {'noun': 'АВТОПОРТРЕТ', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'TNZHEn

20-400 миллисекунд, время зависит от того, насколько БД подгрузила кэш и чем она занята еще.

Ну, неплохо.


---

Попробуем решить еще одну задачу. Возьмем Синтагрус из [Universal Dependencies](https://universaldependencies.org/). Загрузим из него пары слов, связанных синтаксическими отношениями. При этом будем различать расположение главного и зависимого слова - главное справа (->) или главное слева (<-). Далее организуем поиск по этим отношениям, показывая примеры интересующих нас отношений.

Для этого нам придется завести два индекса. В первом будут хранится предложения, во втором - отношения между словами.

**NB**

Вроде бы в одном индексе можно хранить несколько типов документов. Но у меня почему-то не получитлось. Заодно не полчилось организовать хранение потомков и родителей как поддокументов в документе (точнее организовать по ним поиск). Поэтому решение несколько странное.

In [ ]:
es=Elasticsearch()

Так как мы будем проводить аггрегацию по некоторым текстовым полям, то мы добавили к ним `"fielddata": True`.

In [ ]:
#es.indices.delete("syntagrus")
#es.indices.delete("syntagrus_rel")

es.indices.create(index = "syntagrus")
mapit = {"sentences" : {"properties" : {"sentID" : {"type" : "integer"},
                                        "wordPos" : {"type" : "integer"},
                                        "token" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "lemma" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "PoS" : {"type" : "keyword"},
                                        "tag" : {"type" : "keyword"},
                                        "parent" : {"type" : "integer"},
                                        "conn" : {"type" : "text", "fielddata": True}}},
      
         
        }

es.indices.put_mapping(index = "syntagrus", 
                       doc_type = 'sentences', 
                       body = mapit, 
                       include_tupe_name=True)

es.indices.create(index = "syntagrus_rel")
mapit = {"relations" : {"properties" : {"sentID" : {"type" : "integer"},
                                        "conn" : {"type" : "keyword"},
                                        "dir" : {"type" : "keyword"},
                                        "child_PoS" : {"type" : "keyword"},
                                        "parent_PoS" : {"type" : "keyword"},
                                        "parent":{"type": "object", "properties": {
                                        "wordPos" : {"type" : "integer"},
                                        "token" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "lemma" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "PoS" : {"type" : "keyword"},
                                        "tag" : {"type" : "keyword"}
                                        }},
                                        "parent":{"type": "object", "properties": {
                                        "wordPos" : {"type" : "integer"},
                                        "token" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "lemma" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "PoS" : {"type" : "keyword"},
                                        "tag" : {"type" : "keyword"}
                                        }}}
                       }}
es.indices.put_mapping(index = "syntagrus_rel", 
                       doc_type = 'relations', 
                       body = mapit, 
                       include_tupe_name=True)

Читаем все тексты построчно.

In [ ]:
with open("./ru_syntagrus-ud-dev.conllu") as infile:  
    lines=infile.readlines()

Напишем несколько функций: выделяющую предложения из текстов и помещающую данные в индексы.

In [13]:
def readSentence(lines, pos):
    #print("pos=",pos, "line=", lines[pos])
    sent=[]
    while pos<len(lines) and lines[pos]!="\n":
        if lines[pos][0]!='\n' and lines[pos][0]!='#':
            sent.append(lines[pos][:-2].split("\t"))
        pos+=1
    pos+=1
    return sent, pos
    
def putSentence(es, sent, sent_no):
    data=[]
    rels=[]
    for word in sent:
        if '.' in word[0]:
            continue
        data.append({"sentID" : sent_no,
              "wordPos" : word[0],
              "token" : word[1],
              "lemma" : word[2],
              "PoS" : word[3],
              "tag" : word[5],
              "parent" : word[6],
              "conn" : word[7]})
        rels.append({"sentID" : sent_no,
                     "conn" : word[7],
                     "dir" : '->' if word[0]<sent[int(word[6])][0] else '<-',
                     "child_PoS" : word[3],
                     "parent_PoS" : sent[int(word[6])][3],
                     "child":{"properties": {
                         "wordPos" : word[0],
                         "token" : word[1],
                         "lemma" : word[2],
                         "PoS" : word[3],
                         "tag" : word[5]
                     }},
                     "parent":{"properties": {
                         "wordPos" : sent[int(word[6])][0],
                         "token" : sent[int(word[6])][1],
                         "lemma" : sent[int(word[6])][2],
                         "PoS" : sent[int(word[6])][3],
                         "tag" : sent[int(word[6])][5]
                     }}}
        )
        
        #es.index(index="syntagrus", doc_type="sentences", body=data)
        
    bulk(es, rels, index="syntagrus_rel", doc_type='relations')        
    bulk(es, data, index="syntagrus", doc_type='sentences') 

Помещаем данные в индексы. Получается несколько предложений в секунду.

В примере ниже есть первая аггрегация. Чтобы ее задать используются ключевые слова `aggs` или `aggregations`. `"size": 0` необходимо, так как если его не задать, то помимо агрегированной информации будут выдаваться еще и документы, по которым проводилась агрегация.

In [ ]:
res=es.search(index = "syntagrus", body = {"size": 0, "aggs": {"max_id": {"max": {"field": "sentID" }}}})
# Получение максимального номера предложения, уже хранимого в базе.
sent_no=res["aggregations"]["max_id"]["value"]
sent_no=1 if sent_no==None else sent_no+1
sent_no=int(sent_no)

# Пошли по всему файлу. 
pos=0
while pos<len(lines):
    sent, pos=readSentence(lines, pos)
    try:
        putSentence(es, sent, sent_no)
    except:
        pass
    sent_no+=1
    print(sent_no, end="\r")

Теперь посмотрим сколько есть "правых" и "левых" связей. Для этого зададим название поля для агрегации - dirs, поиск будет вестись по терминам - terms, поле поиска - "field" : "dir", сортировка по ключевому слову: "order": {"_term": "asc"}, выдать первые 100 ключевых слов: "size":"100" (по умолчанию - 10).

**NB**
У меня в параллель идет загрузка данных, так что числа будут несколько отличаться от запроса к запросу.

In [30]:
req={
    "size":0,
    "aggs" : {
        "dirs": {"terms" : { "field" : "dir", "size":"100", "order": {"_term": "asc"} }}
    }
}
es.search(index = "syntagrus_rel", body = req)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'aggregations': {'dirs': {'buckets': [{'doc_count': 40974, 'key': '->'},
    {'doc_count': 29490, 'key': '<-'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0}},
 'hits': {'hits': [], 'max_score': 0.0, 'total': 70464},
 'timed_out': False,
 'took': 10}

Как видим, левого ветвления в русском языке больше. Но это ничего не значит. Посмотрим теперь зависимость от части речи главного и зависимого слов. Так как используется вложенная агрегация, то внутри одной мы вставляем другую: внутри агрегации "p_poses" будет агрегация "ch_poses", а в ней - "dirs".

In [31]:
req={
    "size":0,
    "aggs" : {
        "p_poses" : {
            "terms" : { "field" : "parent_PoS", "size":"100", "order": {"_term": "asc"}}, 
            "aggs" : {
                      "ch_poses" : {
                                    "terms" : { "field" : "child_PoS", "size":100, "order": {"_term": "asc"}}, 
                                    "aggs" : {
                                                "dirs": {"terms" : { "field" : "dir", "size":100, "order": {"_term": "asc"} }}
                                    }
                        }
                
            }
            
        }
    }
}
es.search(index = "syntagrus_rel", body = req)


{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'aggregations': {'p_poses': {'buckets': [{'ch_poses': {'buckets': [{'dirs': {'buckets': [{'doc_count': 151,
           'key': '->'},
          {'doc_count': 303, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0},
        'doc_count': 454,
        'key': 'ADJ'},
       {'dirs': {'buckets': [{'doc_count': 83, 'key': '->'},
          {'doc_count': 3, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0},
        'doc_count': 86,
        'key': 'ADP'},
       {'dirs': {'buckets': [{'doc_count': 454, 'key': '->'},
          {'doc_count': 53, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0},
        'doc_count': 507,
        'key': 'ADV'},
       {'dirs': {'buckets': [{'doc_count': 122, 'key': '->'},
          {'doc_count': 46, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum

Первые результаты показаны для прилагательного. И первый его потомок - другое прилагательное. Это немного странно. Посмотрим что за слова друг другу подчиняются и в каком предложении.

При помощи `must` мы говорим БД, что мы объединяем несколько полей, которые должны быть в документе.

In [23]:
req={"query": {"bool": {"must":[{"match": {"child_PoS": "ADJ"}}, {"match": {"parent_PoS": "ADJ"}}]}}}
    
es.search(index = "syntagrus_rel", body = req)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'pZUb_WgBjQGEBxqudDHa',
    '_index': 'syntagrus_rel',
    '_score': 5.7697906,
    '_source': {'child': {'properties': {'PoS': 'ADJ',
       'lemma': 'седой',
       'tag': 'Case=Nom|Degree=Pos|Number=Plur',
       'token': 'седые',
       'wordPos': '13'}},
     'child_PoS': 'ADJ',
     'conn': 'conj',
     'dir': '<-',
     'parent': {'properties': {'PoS': 'ADJ',
       'lemma': 'белый',
       'tag': 'Case=Nom|Degree=Pos|Number=Plur',
       'token': 'белые',
       'wordPos': '10'}},
     'parent_PoS': 'ADJ',
     'sentID': 21},
    '_type': 'relations'},
   {'_id': 'epUb_WgBjQGEBxqutjgC',
    '_index': 'syntagrus_rel',
    '_score': 5.7697906,
    '_source': {'child': {'properties': {'PoS': 'ADJ',
       'lemma': 'требовательный',
       'tag': 'Degree=Pos|Gender=Fem|Number=Sing|Variant=Short',
       'token': 'требовательна',
       'wordPos': '10'}},
     'child_PoS': 'ADJ',
     'c

Для выведенных предложений посмотрим их пару и предложение, в которое они входят.

In [17]:
def printStrangeRelation(ch_PoS, p_PoS, sentID):
    req={"query": {"bool": {"must":[{"match": {"child_PoS": ch_PoS}}, {"match": {"parent_PoS": p_PoS}}, {"match": {"sentID":sentID}}]}}}
    res1=es.search(index = "syntagrus_rel", body = req)
    print(res1["hits"]["hits"][0]["_source"]["child"], res1["hits"]["hits"][0]["_source"]["dir"], res1["hits"]["hits"][0]["_source"]["parent"])
    res=es.search(index = "syntagrus", body = {"query":{"match": {"sentID":sentID}}, "sort":"wordPos", "size":1000})
    print([(r["_source"]["token"], r["_source"]["wordPos"]) for r in res["hits"]["hits"]])

In [24]:
printStrangeRelation('ADJ', 'ADJ', 71)

{'properties': {'wordPos': '8', 'token': 'квалифицированный', 'lemma': 'квалифицированный', 'PoS': 'ADJ', 'tag': 'Case=Nom|Degree=Pos|Gender=Masc|Number=Sing'}} <- {'properties': {'wordPos': '6', 'token': 'честный', 'lemma': 'честный', 'PoS': 'ADJ', 'tag': 'Case=Nom|Degree=Pos|Gender=Masc|Number=Sing'}}
[('Он', '1'), ('сказал', '2'), (',', '3'), ('что', '4'), ('Ефимова', '5'), ('честный', '6'), (',', '7'), ('квалифицированный', '8'), ('работник', '9'), (',', '10'), ('и', '11'), ('выразил', '12'), ('готовность', '13'), ('дать', '14'), ('ей', '15'), ('характеристику', '16'), ('.', '17')]


Интересная логика, но почему нет.